# validator2

This notebook validates materials citations annotation using Darwin core archives downloaded from GGI server. 

### old docs - delete eventually

This notebook controls execution of GGI using **pyautogui**.

It was written to automate downloading XML files for documents from Insects of Guam I and II
as part of a validation process.

This is a fragile hack which probably runs only on my particular setup.

Run in Jupyter lab using the Python 3 kernel. 

The JupyterLab window should display on my laptop, with GGI displaying on the "big screen" (2560 x 1080)

Remember to push to GitHub.

In [5]:
#import pyautogui as pag
import time
#import mysecrets
import pandas as pd
import subprocess
#from bs4 import BeautifulSoup
import re
#import json
from datetime import datetime
import os

In [3]:
def read_dataset_list():
    '''
    Reads ../dataset-list.md and returns a pandas dataframe
    '''
    
    # Read a markdown file, getting the header from the first row and inex from the second column
    # df = pd.read_table('../dataset-list.md', sep="|", header=0, index_col=1, skipinitialspace=True)
    df = pd.read_table('../dataset-list.md', sep="|", header=0, skipinitialspace=True)

    # Drop the left-most and right-most null columns 
    df = df.dropna(axis=1, how='all')

    # Drop the header underline row
    df = df.iloc[1:]  

    # Left-align strings and column headings
    # df = df.style.set_properties(**{'text-align': 'left'})
    # df = df.set_table_styles([dict(selector = 'th', props=[('text-align', 'left')])])

    # Strip whitespace from end of strings
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # Strip whitespace from end of column headers
    df.columns = df.columns.str.strip()

    # Drop datasets with no title - we don't need to process these
    df = df.drop(df[df.title == 'no title'].index)
    return df

# read_dataset_list()

,uuid,status,title
3,FFF07216FFA41642FFBAFFE7FFCA482A,not started,Strepsiptera of Guam
9,A676FD1EF22D3F34FF8F8907FFDAFC58,not started,Halictine Bees from Rota Island
10,FF8CA776FF947F25FF920C4AFFF5FF0F,not started,Aphididae and Aleurodidae Of Guam
11,FFDEFF89B713A955FFD59822FF8CFF82,not started,Isoptera of Guam
12,6137FFB29C68FFD2FFB6585FFF92FFC4,not started,Ciidae of Guam
13,C5751610FFAD3E7CE078FFB1FFCFFF82,not started,Sphingidae Of Guam
14,AD79FFBAEA10FFDBFFFE8726FFBFFFFE,not started,Anthribidae Of Guam
15,FFECFFAC5917FF88FFDE5B1BFFC4FF9B,not started,Rhipiceridae Of Guam
16,FFC7133C860D1213FFF18D1A5E327636,not started,"Trypetidae, Otitidae, Helomyzidae, And Clusiid..."
17,FFF8B07B536B3042BD53344E0F0FFFD7,not started,"Coleoptera, Staphylinidae Of Guam"


In [14]:
!ls -a ~/Desktop/data-mining-insects-of-guam/.git

.	  COMMIT_EDITMSG  FETCH_HEAD  index  objects	  refs
..	  config	  HEAD	      info   ORIG_HEAD
branches  description	  hooks       logs   packed-refs


In [23]:
%%time

# git --git-dir=/path/to/my/repo/.git add .

def update_github():
    x = subprocess.run('git --git-dir=/home/aubrey/Desktop/data-mining-insects-of-guam/.git pull', 
                       shell=True, capture_output=True, text=True)
    assert x.returncode==0, 'git pull failed'

    x = subprocess.run('git --git-dir=/home/aubrey/Desktop/data-mining-insects-of-guam/.git add .', 
                       shell=True, capture_output=True, text=True)
    assert x.returncode==0, 'git add . failed'
        
    x = subprocess.run('git  --git-dir=/home/aubrey/Desktop/data-mining-insects-of-guam/.git commit -m "validator2"', 
                       shell=True, capture_output=True, text=True)
    assert x.returncode==0, 'git commit failed'
        
    x = subprocess.run('git --git-dir=/home/aubrey/Desktop/data-mining-insects-of-guam/.git push', 
                       shell=True, capture_output=True, text=True)
    assert x.returncode==0, 'git push failed'

# update_github()

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.72 µs


In [13]:
def nth_repl(s, sub, repl, n):
    find = s.find(sub)
    # If find is not -1 we have found at least one match for the substring
    i = find != -1
    # loop util we find the nth or we find no match
    while find != -1 and i != n:
        # find + 1 means we start searching from after the last match
        find = s.find(sub, find + 1)
        i += 1
    # If i is equal to n we found nth match so replace
    if i == n:
        return s[:find] + repl + s[find+len(sub):]
    return s

# s = '''
# <tr>
#     <td>alpha</td>
# </tr>
# <tr>
#     <td>beta</td>
# </tr>
# <tr>
#     <td>cappa</td>
# </tr>
# '''
# print(s)
# print()
# s = nth_repl(s, '<tr', '<tr caca', 2)
# print(s)

In [14]:
def create_status_report():
        
    summary_list = []
    for uuid in uuids:
        summary = json.load(open(f'{uuid}_summary.json'))
        summary['uuid'] = uuid
        summary_list.append(summary)
    df_summary = pd.DataFrame(summary_list)
    df_merged = pd.merge(df, df_summary, on='uuid')
    df_merged.drop('status', axis=1, inplace=True)
    #df_merged['validation report'] = f'<a href="{df_merged.uuid}.html">validation report</a>'
    summary_html = df_merged.to_html(index=False)
    
    # Edit so that the table is nicely styled with Bulma
    
    summary_html = summary_html.replace('<table border="1" class="dataframe">', '<table class="table">')
    summary_html = summary_html.replace('<th>', '<th class="has-text-centered">')
    summary_html = summary_html.replace('<td>', '<td class="has-text-centered">')
    
    # Link uuid to validation report
    
    for uuid in uuids:
        summary_html = summary_html.replace(uuid, f'<a href="{uuid}.html">{uuid}</a>')
    
    # Highlight rows which need work
    
    for i, r in df_merged.iterrows():
        if (r['bad collectors'] + r['bad locations'] + r['bad dates']) > 0:
            summary_html = nth_repl(summary_html, '<tr', '<tr class="is-selected"', i+2)  
    
    title_html = f'''
        <p class="title is-1">Insects of Guam Datamining Project</p>
        <p class="subtitle is-3">Status report generated by GGIbot at {datetime.utcnow()} UTC</p>
        <p>Highlited documents failed validation. Click on the **uuid** to see the validation report.</p>
        '''

    html = f'''
        <html>
            <header>
                <meta charset="utf-8">
                <meta name="viewport" content="width=device-width, initial-scale=1">
                <title>mat_cit_chk</title>
                <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bulma@0.9.3/css/bulma.min.css">
            </header>
            <body>
                <section class="section">
                    <div class="container">
                        {title_html}
                        {summary_html}
                    </div>
                </section>
            </body>
        </html>        
        '''
    with open('status_report.html', 'w') as f:
        f.write(html) 
        
# create_status_report()

In [15]:
%%time

# MAIN

df = read_dataset_list()

# lets process the first 15 datasets

uuids = list(df.iloc[0:15].uuid.values)
# open_GGI()
# login()
for uuid in uuids:
    print(uuid)
#     load_doc(uuid)
#     export_xml(uuid)
#     process_doc(uuid)
#     close_doc()
# create_status_report()

print('Updating GitHub')
update_github()
        
print('FINISHED')
# print('Please update git repo for ~/Desktop/data-mining-insects-of-guam')
print()

Updating GitHub
0
0
0
0
FINISHED

CPU times: user 4.35 s, sys: 895 ms, total: 5.25 s
Wall time: 19min 21s
